# Convert from JSON to CSV

This is so we can tabulate the data and upload to a database for later querying.

In [69]:
import json
import pandas as pd
from flatten_json import flatten
from datetime import timedelta, datetime
from pprint import pprint

In [130]:
FILENAME = "film"
# mica, ceramic, inductors, film, aluminum_electrolytic

In [131]:
def delete_keys_from_dict(d, to_delete):
    if isinstance(to_delete, str):
        to_delete = [to_delete]
    if isinstance(d, dict):
        for single_to_delete in set(to_delete):
            if single_to_delete in d:
                del d[single_to_delete]
        for k, v in d.items():
            delete_keys_from_dict(v, to_delete)
    elif isinstance(d, list):
        for i in d:
            delete_keys_from_dict(i, to_delete)

## Preprocess the Data
Currently we have data in the JSON file that is stored as an array, and we want to flatten it based on the type of data it stores. Specifically, we want to flatten the "specs" array.

In [132]:
with(open(f'{FILENAME}.json', "r", encoding='utf-8')) as f:
    data = json.load(f)

In [133]:
len(data['data']['search']['results'])
pprint(data['data']['search']['results'][2])

{'__typename': 'PartResult',
 '_cache_id': 'US-USD-71518146',
 'description': 'Cap Film 0.1uF 630V PP 5%( 17.5 X 5 X 10.5mm) Radial Plastic '
                'Rectangular Can 15mm 105C Ammo',
 'part': {'__typename': 'Part',
          '_cache_id': 'US-USD-71518146',
          'best_datasheet': {'__typename': 'Document',
                             'url': 'http://datasheet.octopart.com/ECWFE2J104J-Panasonic-datasheet-64277301.pdf'},
          'best_image': {'__typename': 'Image',
                         'url': 'https://sigma.octopart.com/65561776/image/Panasonic-ECWFE2J104J.jpg'},
          'category': {'__typename': 'Category', 'id': '6333'},
          'counts': {'companion_products': 0,
                     'datasheets': 4,
                     'descriptions': 5,
                     'images': 4,
                     'reference_designs': 0,
                     'similar_parts': 12,
                     'specs': 20},
          'descriptions': [{'__typename': 'Description',
           

In [134]:
delete_keys_from_dict(data, ['__typename', 'best_datasheet', 'best_image', 'manufacturer_url'])

for result in data['data']['search']['results']:
    # Go through each component and flatten the specs dictionary.
    spec_json = {}
    for spec in result['part']['specs']:
        title = spec['attribute']['shortname']
        spec['attribute']['display_value'] = spec['display_value']
        spec = spec['attribute']
        del spec['shortname']
        del spec['name']
        spec_json[title] = spec
    result['part']['specs'] = spec_json

    del result['part']['_cache_id']
    del result['part']['descriptions']
    del result['part']['counts']

In [135]:
len(data['data']['search']['results'])
pprint(data['data']['search']['results'][5])

{'_cache_id': 'US-USD-21765912',
 'description': 'Cap Film Suppression X2 0.1uF 630VDC/275VAC Paper 20% (27 X 8 '
                'X 17mm) Radial 22.5mm 110C Bag',
 'part': {'category': {'id': '6333'},
          'id': '21765912',
          'manufacturer': {'id': '212', 'is_verified': False, 'name': 'KEMET'},
          'median_price_1000': {'_cache_id': 'US-USD-21765912-1000-1.043495',
                                'converted_currency': 'USD',
                                'converted_price': 1.043495},
          'mpn': 'PME271MD6100MR30',
          'specs': {'capacitance': {'display_value': '100 nF', 'id': '548'},
                    'case_package': {'display_value': 'Radial', 'id': '842'},
                    'depth': {'display_value': '8 mm', 'id': '291'},
                    'dielectricmaterial': {'display_value': 'Paper',
                                           'id': '834'},
                    'height': {'display_value': '17 mm', 'id': '468'},
                    'height_sea

In [144]:
# https://developers.cloudflare.com/analytics/graphql-api/tutorials/export-graphql-to-csv/
def convert_to_csv(flat_json):
    df = pd.DataFrame(flat_json, dtype ='str')
    file = f"csv/{FILENAME}.csv"

    # Remove all columns that have "__typename" in the name.
    df.drop(list(df.filter(regex = '__typename')), axis = 1, inplace = True)
    df.to_csv(file, index=False)

def create_sql_query(flat_json, name=FILENAME):
    unique_keys = {key:type(dic[key]) for dic in flat_json for key in dic.keys()}
    pg_dict = {}
    for k, v in unique_keys.items():
#         if v == bool:
#             pg_dict[k] = "BOOL"
#         elif v == int:
#             pg_dict[k] = "INT"
#         elif v == float:
#             pg_dict[k] = "FLOAT8"
#         else:
            pg_dict[k] = "VARCHAR"

    i = 0
    for k, v in pg_dict.items():
        if i == 0:
            print(f"CREATE TABLE {name} (")
            print(f"\t{k} {v},")
        elif i == len(pg_dict.values())-1:
            print(f"\t{k} {v}\n);")
        else:
            print(f"\t{k} {v},")
        i += 1

In [137]:
flat = [flatten(d) for d in data['data']['search']['results']]

In [138]:
convert_to_csv(flat)

In [139]:
create_sql_query(flat)

CREATE TABLE film (
	_cache_id VARCHAR,
	description VARCHAR,
	part_category_id VARCHAR,
	part_id VARCHAR,
	part_manufacturer_id VARCHAR,
	part_manufacturer_is_verified VARCHAR,
	part_manufacturer_name VARCHAR,
	part_median_price_1000__cache_id VARCHAR,
	part_median_price_1000_converted_currency VARCHAR,
	part_median_price_1000_converted_price VARCHAR,
	part_mpn VARCHAR,
	part_specs_capacitance_id VARCHAR,
	part_specs_capacitance_display_value VARCHAR,
	part_specs_case_package_id VARCHAR,
	part_specs_case_package_display_value VARCHAR,
	part_specs_depth_id VARCHAR,
	part_specs_depth_display_value VARCHAR,
	part_specs_dielectric_id VARCHAR,
	part_specs_dielectric_display_value VARCHAR,
	part_specs_dielectricmaterial_id VARCHAR,
	part_specs_dielectricmaterial_display_value VARCHAR,
	part_specs_height_id VARCHAR,
	part_specs_height_display_value VARCHAR,
	part_specs_height_seated_max__id VARCHAR,
	part_specs_height_seated_max__display_value VARCHAR,
	part_specs_leaddiameter_id VARCHAR,
	p

In [146]:
files = ["mica", "ceramic", "inductors", "film", "aluminum_electrolytic"]
for file in files:
    with(open(f'{file}.json', "r", encoding='utf-8')) as f:
        data = json.load(f)

        delete_keys_from_dict(data, ['__typename', 'best_datasheet', 'best_image', 'manufacturer_url'])

        for result in data['data']['search']['results']:
            # Go through each component and flatten the specs dictionary.
            spec_json = {}
            for spec in result['part']['specs']:
                title = spec['attribute']['shortname']
                spec['attribute']['display_value'] = spec['display_value']
                spec = spec['attribute']
                del spec['shortname']
                del spec['name']
                spec_json[title] = spec
            result['part']['specs'] = spec_json

            del result['part']['_cache_id']
            del result['part']['descriptions']
            del result['part']['counts']
        
        flat = [flatten(d) for d in data['data']['search']['results']]
        create_sql_query(flat, file)

CREATE TABLE mica (
	_cache_id VARCHAR,
	description VARCHAR,
	part_category_id VARCHAR,
	part_id VARCHAR,
	part_manufacturer_id VARCHAR,
	part_manufacturer_is_verified VARCHAR,
	part_manufacturer_name VARCHAR,
	part_median_price_1000__cache_id VARCHAR,
	part_median_price_1000_converted_currency VARCHAR,
	part_median_price_1000_converted_price VARCHAR,
	part_mpn VARCHAR,
	part_specs_capacitance_id VARCHAR,
	part_specs_capacitance_display_value VARCHAR,
	part_specs_case_package_id VARCHAR,
	part_specs_case_package_display_value VARCHAR,
	part_specs_depth_id VARCHAR,
	part_specs_depth_display_value VARCHAR,
	part_specs_dielectricmaterial_id VARCHAR,
	part_specs_dielectricmaterial_display_value VARCHAR,
	part_specs_features_id VARCHAR,
	part_specs_features_display_value VARCHAR,
	part_specs_height_id VARCHAR,
	part_specs_height_display_value VARCHAR,
	part_specs_height_seated_max__id VARCHAR,
	part_specs_height_seated_max__display_value VARCHAR,
	part_specs_leaddiameter_id VARCHAR,
	part_

CREATE TABLE inductors (
	_cache_id VARCHAR,
	description VARCHAR,
	part_category_id VARCHAR,
	part_id VARCHAR,
	part_manufacturer_id VARCHAR,
	part_manufacturer_is_verified VARCHAR,
	part_manufacturer_name VARCHAR,
	part_median_price_1000__cache_id VARCHAR,
	part_median_price_1000_converted_currency VARCHAR,
	part_median_price_1000_converted_price VARCHAR,
	part_mpn VARCHAR,
	part_specs_case_package_id VARCHAR,
	part_specs_case_package_display_value VARCHAR,
	part_specs_casecode_imperial__id VARCHAR,
	part_specs_casecode_imperial__display_value VARCHAR,
	part_specs_casecode_metric__id VARCHAR,
	part_specs_casecode_metric__display_value VARCHAR,
	part_specs_composition_id VARCHAR,
	part_specs_composition_display_value VARCHAR,
	part_specs_corematerial_id VARCHAR,
	part_specs_corematerial_display_value VARCHAR,
	part_specs_currentrating_id VARCHAR,
	part_specs_currentrating_display_value VARCHAR,
	part_specs_dcresistance_dcr__id VARCHAR,
	part_specs_dcresistance_dcr__display_value VARCH

CREATE TABLE film (
	_cache_id VARCHAR,
	description VARCHAR,
	part_category_id VARCHAR,
	part_id VARCHAR,
	part_manufacturer_id VARCHAR,
	part_manufacturer_is_verified VARCHAR,
	part_manufacturer_name VARCHAR,
	part_median_price_1000__cache_id VARCHAR,
	part_median_price_1000_converted_currency VARCHAR,
	part_median_price_1000_converted_price VARCHAR,
	part_mpn VARCHAR,
	part_specs_capacitance_id VARCHAR,
	part_specs_capacitance_display_value VARCHAR,
	part_specs_case_package_id VARCHAR,
	part_specs_case_package_display_value VARCHAR,
	part_specs_depth_id VARCHAR,
	part_specs_depth_display_value VARCHAR,
	part_specs_dielectric_id VARCHAR,
	part_specs_dielectric_display_value VARCHAR,
	part_specs_dielectricmaterial_id VARCHAR,
	part_specs_dielectricmaterial_display_value VARCHAR,
	part_specs_height_id VARCHAR,
	part_specs_height_display_value VARCHAR,
	part_specs_height_seated_max__id VARCHAR,
	part_specs_height_seated_max__display_value VARCHAR,
	part_specs_leaddiameter_id VARCHAR,
	p

In [22]:
from sqlalchemy import create_engine
import psycopg2 
import io

def convert_to_csv_2(flat_json):
    
    df = pd.DataFrame(flat_json, dtype ='str')
    print(df.memory_usage())
    host = "ec2-34-233-115-14.compute-1.amazonaws.com"
    port = 5432
    database = "dfu56m15dkhh46"
    user = "pgyrjmstmyerfk"
    password = "228fcbba14e9d2bf362fcaa29cabe1106cc8dba00605f45ee25e810194309fd4"

    conn_str = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(conn_str)
    
#     df.to_sql('ceramic', engine, index=False, method='multi', if_exists='replace')
    
    

#     df.head(0).to_sql('ceramic', engine, if_exists='replace',index=False) #drops old table and creates new empty table

#     conn = engine.raw_connection()
#     conn.set_client_encoding('UTF8')
#     cur = conn.cursor()
#     output = io.StringIO()
#     df.to_csv(output, encoding='utf-8', sep='\t', header=False, index=False)
#     output.seek(0)
#     contents = output.getvalue()
#     cur.copy_from(output, 'ceramic', null="") # null values become ''
#     conn.commit()
    
#     file = f"csv/{FILENAME}.csv"

    # Remove all columns that have "__typename" in the name.
#     df.drop(list(df.filter(regex = '__typename')), axis = 1, inplace = True)
#     df.to_csv(file, index=False)

flat = [flatten(d) for d in data['data']['search']['results']]
convert_to_csv_2(flat)

Index                                               128
_cache_id                                       1802376
description                                     1802376
part_category_id                                1802376
part_id                                         1802376
                                                 ...   
part_specs_inductancetolerance_display_value    1802376
part_specs_life_hours__id                       1802376
part_specs_life_hours__name                     1802376
part_specs_life_hours__shortname                1802376
part_specs_life_hours__display_value            1802376
Length: 497, dtype: int64


In [142]:
import pandas as pd
  
df = pd.concat(
    map(pd.read_csv, ['csv/aluminum_electrolytic.csv', 
                      'csv/ceramic.csv', 
                      'csv/film.csv', 
                      'csv/mica.csv', 
                      'csv/inductors.csv']), ignore_index=True)
df = df.astype(str)
df.to_csv("csv/all.csv", index=False)

/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_81613/677482548.py:3: DtypeWarning: Columns (101,105,107,111,117,119,123,127,129,131,133,135,137,139,141,143,145,147,149,151) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(
/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_81613/677482548.py:3: DtypeWarning: Columns (14,20,22,24,30,32,36,38,40,46,50,52,54,62,64,66,68,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,121,129,131,133,135,139,141,143,145,147,149,151,153,155,157,159,161,163,167,169,171,173,177,179,181,183,185,187,189,191,193,195,197,199,201,203,205,207,209,211,213,215,217,219,221,223,225,227,229,231,233,235,237,239,241,243,245,247,251,253) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(
/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_81613/677482548.py:3: DtypeWarning: Columns (82,88,90,99,103,107,109,115,117,119,121,123,125,127,

In [3]:
cols = "Unnamed: 0,_cache_id,description,part__cache_id,part_best_datasheet_url,part_best_image_url,part_category_id,part_counts_companion_products,part_counts_datasheets,part_counts_descriptions,part_counts_images,part_counts_reference_designs,part_counts_similar_parts,part_counts_specs,part_descriptions_0_text,part_descriptions_1_text,part_descriptions_2_text,part_descriptions_3_text,part_id,part_manufacturer_id,part_manufacturer_is_verified,part_manufacturer_name,part_manufacturer_url,part_median_price_1000__cache_id,part_median_price_1000_converted_currency,part_median_price_1000_converted_price,part_mpn,part_specs_capacitance_attribute_id,part_specs_capacitance_attribute_name,part_specs_capacitance_attribute_shortname,part_specs_capacitance_display_value,part_specs_case_package_attribute_id,part_specs_case_package_attribute_name,part_specs_case_package_attribute_shortname,part_specs_case_package_display_value,part_specs_diameter_attribute_id,part_specs_diameter_attribute_name,part_specs_diameter_attribute_shortname,part_specs_diameter_display_value,part_specs_esr_equivalentseriesresistance__attribute_id,part_specs_esr_equivalentseriesresistance__attribute_name,part_specs_esr_equivalentseriesresistance__attribute_shortname,part_specs_esr_equivalentseriesresistance__display_value,part_specs_height_attribute_id,part_specs_height_attribute_name,part_specs_height_attribute_shortname,part_specs_height_display_value,part_specs_height_seated_max__attribute_id,part_specs_height_seated_max__attribute_name,part_specs_height_seated_max__attribute_shortname,part_specs_height_seated_max__display_value,part_specs_leadpitch_attribute_id,part_specs_leadpitch_attribute_name,part_specs_leadpitch_attribute_shortname,part_specs_leadpitch_display_value,part_specs_length_attribute_id,part_specs_length_attribute_name,part_specs_length_attribute_shortname,part_specs_length_display_value,part_specs_life_hours__attribute_id,part_specs_life_hours__attribute_name,part_specs_life_hours__attribute_shortname,part_specs_life_hours__display_value,part_specs_maxoperatingtemperature_attribute_id,part_specs_maxoperatingtemperature_attribute_name,part_specs_maxoperatingtemperature_attribute_shortname,part_specs_maxoperatingtemperature_display_value,part_specs_minoperatingtemperature_attribute_id,part_specs_minoperatingtemperature_attribute_name,part_specs_minoperatingtemperature_attribute_shortname,part_specs_minoperatingtemperature_display_value,part_specs_mount_attribute_id,part_specs_mount_attribute_name,part_specs_mount_attribute_shortname,part_specs_mount_display_value,part_specs_packaging_attribute_id,part_specs_packaging_attribute_name,part_specs_packaging_attribute_shortname,part_specs_packaging_display_value,part_specs_reachsvhc_attribute_id,part_specs_reachsvhc_attribute_name,part_specs_reachsvhc_attribute_shortname,part_specs_reachsvhc_display_value,part_specs_ripplecurrent_attribute_id,part_specs_ripplecurrent_attribute_name,part_specs_ripplecurrent_attribute_shortname,part_specs_ripplecurrent_display_value,part_specs_rohs_attribute_id,part_specs_rohs_attribute_name,part_specs_rohs_attribute_shortname,part_specs_rohs_display_value,part_specs_termination_attribute_id,part_specs_termination_attribute_name,part_specs_termination_attribute_shortname,part_specs_termination_display_value,part_specs_tolerance_attribute_id,part_specs_tolerance_attribute_name,part_specs_tolerance_attribute_shortname,part_specs_tolerance_display_value,part_specs_voltagerating_attribute_id,part_specs_voltagerating_attribute_name,part_specs_voltagerating_attribute_shortname,part_specs_voltagerating_display_value,part_specs_voltagerating_dc__attribute_id,part_specs_voltagerating_dc__attribute_name,part_specs_voltagerating_dc__attribute_shortname,part_specs_voltagerating_dc__display_value,part_specs_workingvoltage_attribute_id,part_specs_workingvoltage_attribute_name,part_specs_workingvoltage_attribute_shortname,part_specs_workingvoltage_display_value,part_specs_dielectricmaterial_attribute_id,part_specs_dielectricmaterial_attribute_name,part_specs_dielectricmaterial_attribute_shortname,part_specs_dielectricmaterial_display_value,part_specs_numberofpins_attribute_id,part_specs_numberofpins_attribute_name,part_specs_numberofpins_attribute_shortname,part_specs_numberofpins_display_value,part_descriptions_4_text,part_specs_leakagecurrent_attribute_id,part_specs_leakagecurrent_attribute_name,part_specs_leakagecurrent_attribute_shortname,part_specs_leakagecurrent_display_value,part_best_datasheet,part_best_image,part_specs_polarity_attribute_id,part_specs_polarity_attribute_name,part_specs_polarity_attribute_shortname,part_specs_polarity_display_value,part_descriptions,part_median_price_1000,part_specs_leadfree_attribute_id,part_specs_leadfree_attribute_name,part_specs_leadfree_attribute_shortname,part_specs_leadfree_display_value,part_specs_leaddiameter_attribute_id,part_specs_leaddiameter_attribute_name,part_specs_leaddiameter_attribute_shortname,part_specs_leaddiameter_display_value,part_specs_dissipationfactor_attribute_id,part_specs_dissipationfactor_attribute_name,part_specs_dissipationfactor_attribute_shortname,part_specs_dissipationfactor_display_value,part_specs_testfrequency_attribute_id,part_specs_testfrequency_attribute_name,part_specs_testfrequency_attribute_shortname,part_specs_testfrequency_display_value,part_specs_contactplating_attribute_id,part_specs_contactplating_attribute_name,part_specs_contactplating_attribute_shortname,part_specs_contactplating_display_value,part_specs_voltagerating_ac__attribute_id,part_specs_voltagerating_ac__attribute_name,part_specs_voltagerating_ac__attribute_shortname,part_specs_voltagerating_ac__display_value,part_specs_radiationhardening_attribute_id,part_specs_radiationhardening_attribute_name,part_specs_radiationhardening_attribute_shortname,part_specs_radiationhardening_display_value,part_specs_type_attribute_id,part_specs_type_attribute_name,part_specs_type_attribute_shortname,part_specs_type_display_value,part_specs_features_attribute_id,part_specs_features_attribute_name,part_specs_features_attribute_shortname,part_specs_features_display_value,part_specs_weight_attribute_id,part_specs_weight_attribute_name,part_specs_weight_attribute_shortname,part_specs_weight_display_value,part_specs_lifecyclestatus_attribute_id,part_specs_lifecyclestatus_attribute_name,part_specs_lifecyclestatus_attribute_shortname,part_specs_lifecyclestatus_display_value,part_specs_ripplecurrent_ac__attribute_id,part_specs_ripplecurrent_ac__attribute_name,part_specs_ripplecurrent_ac__attribute_shortname,part_specs_ripplecurrent_ac__display_value,part_specs_resistance_attribute_id,part_specs_resistance_attribute_name,part_specs_resistance_attribute_shortname,part_specs_resistance_display_value,part_specs_leadlength_attribute_id,part_specs_leadlength_attribute_name,part_specs_leadlength_attribute_shortname,part_specs_leadlength_display_value,part_descriptions_5_text,part_specs_scheduleB_attribute_id,part_specs_scheduleB_attribute_name,part_specs_scheduleB_attribute_shortname,part_specs_scheduleB_display_value,part_specs_voltage_attribute_id,part_specs_voltage_attribute_name,part_specs_voltage_attribute_shortname,part_specs_voltage_display_value,part_specs_impedance_attribute_id,part_specs_impedance_attribute_name,part_specs_impedance_attribute_shortname,part_specs_impedance_display_value,part_specs_lead_basestyle_attribute_id,part_specs_lead_basestyle_attribute_name,part_specs_lead_basestyle_attribute_shortname,part_specs_lead_basestyle_display_value,part_specs_width_attribute_id,part_specs_width_attribute_name,part_specs_width_attribute_shortname,part_specs_width_display_value,part_specs_holediameter_attribute_id,part_specs_holediameter_attribute_name,part_specs_holediameter_attribute_shortname,part_specs_holediameter_display_value,part_specs_material_attribute_id,part_specs_material_attribute_name,part_specs_material_attribute_shortname,part_specs_material_display_value,part_specs_inductancetolerance_attribute_id,part_specs_inductancetolerance_attribute_name,part_specs_inductancetolerance_attribute_shortname,part_specs_inductancetolerance_display_value,part_descriptions_6_text,part_specs_outsidediameter_attribute_id,part_specs_outsidediameter_attribute_name,part_specs_outsidediameter_attribute_shortname,part_specs_outsidediameter_display_value,part_specs_depth_attribute_id,part_specs_depth_attribute_name,part_specs_depth_attribute_shortname,part_specs_depth_display_value,part_specs_insulationmaterial_attribute_id,part_specs_insulationmaterial_attribute_name,part_specs_insulationmaterial_attribute_shortname,part_specs_insulationmaterial_display_value,part_specs_casecode_imperial__attribute_id,part_specs_casecode_imperial__attribute_name,part_specs_casecode_imperial__attribute_shortname,part_specs_casecode_imperial__display_value,part_specs_casecode_metric__attribute_id,part_specs_casecode_metric__attribute_name,part_specs_casecode_metric__attribute_shortname,part_specs_casecode_metric__display_value,part_specs_mfgpackage_id_attribute_id,part_specs_mfgpackage_id_attribute_name,part_specs_mfgpackage_id_attribute_shortname,part_specs_mfgpackage_id_display_value,part_descriptions_7_text,part_specs_pitch_attribute_id,part_specs_pitch_attribute_name,part_specs_pitch_attribute_shortname,part_specs_pitch_display_value,part_specs_manufacturerlifecyclestatus_attribute_id,part_specs_manufacturerlifecyclestatus_attribute_name,part_specs_manufacturerlifecyclestatus_attribute_shortname,part_specs_manufacturerlifecyclestatus_display_value,part_specs_maxlength_attribute_id,part_specs_maxlength_attribute_name,part_specs_maxlength_attribute_shortname,part_specs_maxlength_display_value,part_specs_qfactor_attribute_id,part_specs_qfactor_attribute_name,part_specs_qfactor_attribute_shortname,part_specs_qfactor_display_value,part_specs_maxvoltagerating_dc__attribute_id,part_specs_maxvoltagerating_dc__attribute_name,part_specs_maxvoltagerating_dc__attribute_shortname,part_specs_maxvoltagerating_dc__display_value,part_specs_threadsize_attribute_id,part_specs_threadsize_attribute_name,part_specs_threadsize_attribute_shortname,part_specs_threadsize_display_value,part_specs_color_attribute_id,part_specs_color_attribute_name,part_specs_color_attribute_shortname,part_specs_color_display_value,part_specs_forwardcurrent_attribute_id,part_specs_forwardcurrent_attribute_name,part_specs_forwardcurrent_attribute_shortname,part_specs_forwardcurrent_display_value,part_specs_forwardvoltage_attribute_id,part_specs_forwardvoltage_attribute_name,part_specs_forwardvoltage_attribute_shortname,part_specs_forwardvoltage_display_value,part_specs_illuminationcolor_attribute_id,part_specs_illuminationcolor_attribute_name,part_specs_illuminationcolor_attribute_shortname,part_specs_illuminationcolor_display_value,part_specs_lensstyle_attribute_id,part_specs_lensstyle_attribute_name,part_specs_lensstyle_attribute_shortname,part_specs_lensstyle_display_value,part_specs_luminousintensity_attribute_id,part_specs_luminousintensity_attribute_name,part_specs_luminousintensity_attribute_shortname,part_specs_luminousintensity_display_value,part_specs_dielectric_attribute_id,part_specs_dielectric_attribute_name,part_specs_dielectric_attribute_shortname,part_specs_dielectric_display_value,part_descriptions_8_text,part_specs_current_attribute_id,part_specs_current_attribute_name,part_specs_current_attribute_shortname,part_specs_current_display_value,part_specs_wire_cablediameter_attribute_id,part_specs_wire_cablediameter_attribute_name,part_specs_wire_cablediameter_attribute_shortname,part_specs_wire_cablediameter_display_value,part_specs_orientation_attribute_id,part_specs_orientation_attribute_name,part_specs_orientation_attribute_shortname,part_specs_orientation_display_value,part_specs_switchingvoltage_attribute_id,part_specs_switchingvoltage_attribute_name,part_specs_switchingvoltage_attribute_shortname,part_specs_switchingvoltage_display_value,part_specs_contactmaterial_attribute_id,part_specs_contactmaterial_attribute_name,part_specs_contactmaterial_attribute_shortname,part_specs_contactmaterial_display_value,part_specs_temperaturecoefficient_attribute_id,part_specs_temperaturecoefficient_attribute_name,part_specs_temperaturecoefficient_attribute_shortname,part_specs_temperaturecoefficient_display_value,part_specs_thickness_attribute_id,part_specs_thickness_attribute_name,part_specs_thickness_attribute_shortname,part_specs_thickness_display_value,part_specs_insulationresistance_attribute_id,part_specs_insulationresistance_attribute_name,part_specs_insulationresistance_attribute_shortname,part_specs_insulationresistance_display_value,part_specs_composition_attribute_id,part_specs_composition_attribute_name,part_specs_composition_attribute_shortname,part_specs_composition_display_value,part_specs_halogenfree_attribute_id,part_specs_halogenfree_attribute_name,part_specs_halogenfree_attribute_shortname,part_specs_halogenfree_display_value,part_specs_ratings_attribute_id,part_specs_ratings_attribute_name,part_specs_ratings_attribute_shortname,part_specs_ratings_display_value,part_descriptions_9_text,part_descriptions_10_text,part_descriptions_11_text,part_specs_maxthickness_attribute_id,part_specs_maxthickness_attribute_name,part_specs_maxthickness_attribute_shortname,part_specs_maxthickness_display_value,part_specs_maxwidth_attribute_id,part_specs_maxwidth_attribute_name,part_specs_maxwidth_attribute_shortname,part_specs_maxwidth_display_value,part_specs_minlength_attribute_id,part_specs_minlength_attribute_name,part_specs_minlength_attribute_shortname,part_specs_minlength_display_value,part_specs_minthickness_attribute_id,part_specs_minthickness_attribute_name,part_specs_minthickness_attribute_shortname,part_specs_minthickness_display_value,part_specs_minwidth_attribute_id,part_specs_minwidth_attribute_name,part_specs_minwidth_attribute_shortname,part_specs_minwidth_display_value,part_specs_insidediameter_attribute_id,part_specs_insidediameter_attribute_name,part_specs_insidediameter_attribute_shortname,part_specs_insidediameter_display_value,part_specs_circuittype_attribute_id,part_specs_circuittype_attribute_name,part_specs_circuittype_attribute_shortname,part_specs_circuittype_display_value,part_specs_numberofcapacitors_attribute_id,part_specs_numberofcapacitors_attribute_name,part_specs_numberofcapacitors_attribute_shortname,part_specs_numberofcapacitors_display_value,part_specs_numberofelements_attribute_id,part_specs_numberofelements_attribute_name,part_specs_numberofelements_attribute_shortname,part_specs_numberofelements_display_value,part_specs_numberofterminals_attribute_id,part_specs_numberofterminals_attribute_name,part_specs_numberofterminals_attribute_shortname,part_specs_numberofterminals_display_value,part_specs_terminalpitch_attribute_id,part_specs_terminalpitch_attribute_name,part_specs_terminalpitch_attribute_shortname,part_specs_terminalpitch_display_value,part_specs_failurerate_attribute_id,part_specs_failurerate_attribute_name,part_specs_failurerate_attribute_shortname,part_specs_failurerate_display_value,part_specs_numberofcircuits_attribute_id,part_specs_numberofcircuits_attribute_name,part_specs_numberofcircuits_attribute_shortname,part_specs_numberofcircuits_display_value,part_specs_responsetime_attribute_id,part_specs_responsetime_attribute_name,part_specs_responsetime_attribute_shortname,part_specs_responsetime_display_value,part_specs_frequency_attribute_id,part_specs_frequency_attribute_name,part_specs_frequency_attribute_shortname,part_specs_frequency_display_value,part_specs_maxjunctiontemperature_attribute_id,part_specs_maxjunctiontemperature_attribute_name,part_specs_maxjunctiontemperature_attribute_shortname,part_specs_maxjunctiontemperature_display_value,part_specs_maxsupplyvoltage_attribute_id,part_specs_maxsupplyvoltage_attribute_name,part_specs_maxsupplyvoltage_attribute_shortname,part_specs_maxsupplyvoltage_display_value,part_specs_minsupplyvoltage_attribute_id,part_specs_minsupplyvoltage_attribute_name,part_specs_minsupplyvoltage_attribute_shortname,part_specs_minsupplyvoltage_display_value,part_specs_nominalsupplycurrent_attribute_id,part_specs_nominalsupplycurrent_attribute_name,part_specs_nominalsupplycurrent_attribute_shortname,part_specs_nominalsupplycurrent_display_value,part_specs_elv_attribute_id,part_specs_elv_attribute_name,part_specs_elv_attribute_shortname,part_specs_elv_display_value,part_specs_selfresonantfrequency_attribute_id,part_specs_selfresonantfrequency_attribute_name,part_specs_selfresonantfrequency_attribute_shortname,part_specs_selfresonantfrequency_display_value,part_specs_powerrating_attribute_id,part_specs_powerrating_attribute_name,part_specs_powerrating_attribute_shortname,part_specs_powerrating_display_value,part_specs_corearchitecture_attribute_id,part_specs_corearchitecture_attribute_name,part_specs_corearchitecture_attribute_shortname,part_specs_corearchitecture_display_value,part_specs_interface_attribute_id,part_specs_interface_attribute_name,part_specs_interface_attribute_shortname,part_specs_interface_display_value,part_specs_memorysize_attribute_id,part_specs_memorysize_attribute_name,part_specs_memorysize_attribute_shortname,part_specs_memorysize_display_value,part_specs_memorytype_attribute_id,part_specs_memorytype_attribute_name,part_specs_memorytype_attribute_shortname,part_specs_memorytype_display_value,part_specs_numberofi_os_attribute_id,part_specs_numberofi_os_attribute_name,part_specs_numberofi_os_attribute_shortname,part_specs_numberofi_os_display_value,part_specs_oscillatortype_attribute_id,part_specs_oscillatortype_attribute_name,part_specs_oscillatortype_attribute_shortname,part_specs_oscillatortype_display_value,part_specs_peripherals_attribute_id,part_specs_peripherals_attribute_name,part_specs_peripherals_attribute_shortname,part_specs_peripherals_display_value,part_specs_numberofleds_attribute_id,part_specs_numberofleds_attribute_name,part_specs_numberofleds_attribute_shortname,part_specs_numberofleds_display_value,part_specs_viewingangle_attribute_id,part_specs_viewingangle_attribute_name,part_specs_viewingangle_attribute_shortname,part_specs_viewingangle_display_value,part_specs_frequencytolerance_attribute_id,part_specs_frequencytolerance_attribute_name,part_specs_frequencytolerance_attribute_shortname,part_specs_frequencytolerance_display_value,part_specs_loadcapacitance_attribute_id,part_specs_loadcapacitance_attribute_name,part_specs_loadcapacitance_attribute_shortname,part_specs_loadcapacitance_display_value,part_specs_currentrating_attribute_id,part_specs_currentrating_attribute_name,part_specs_currentrating_attribute_shortname,part_specs_currentrating_display_value,part_specs_dcresistance_dcr__attribute_id,part_specs_dcresistance_dcr__attribute_name,part_specs_dcresistance_dcr__attribute_shortname,part_specs_dcresistance_dcr__display_value,part_specs_outputcurrent_attribute_id,part_specs_outputcurrent_attribute_name,part_specs_outputcurrent_attribute_shortname,part_specs_outputcurrent_display_value,part_specs_maxcurrentrating_attribute_id,part_specs_maxcurrentrating_attribute_name,part_specs_maxcurrentrating_attribute_shortname,part_specs_maxcurrentrating_display_value,part_specs_maxdccurrent_attribute_id,part_specs_maxdccurrent_attribute_name,part_specs_maxdccurrent_attribute_shortname,part_specs_maxdccurrent_display_value,part_specs_maxoutputcurrent_attribute_id,part_specs_maxoutputcurrent_attribute_name,part_specs_maxoutputcurrent_attribute_shortname,part_specs_maxoutputcurrent_display_value,part_specs_mininputvoltage_attribute_id,part_specs_mininputvoltage_attribute_name,part_specs_mininputvoltage_attribute_shortname,part_specs_mininputvoltage_display_value,part_specs_outputvoltage_attribute_id,part_specs_outputvoltage_attribute_name,part_specs_outputvoltage_attribute_shortname,part_specs_outputvoltage_display_value,part_specs_accuracy_attribute_id,part_specs_accuracy_attribute_name,part_specs_accuracy_attribute_shortname,part_specs_accuracy_display_value,part_specs_characterheight_attribute_id,part_specs_characterheight_attribute_name,part_specs_characterheight_attribute_shortname,part_specs_characterheight_display_value,part_specs_bandwidth_attribute_id,part_specs_bandwidth_attribute_name,part_specs_bandwidth_attribute_shortname,part_specs_bandwidth_display_value,part_specs_nominalsupplyvoltage_dc__attribute_id,part_specs_nominalsupplyvoltage_dc__attribute_name,part_specs_nominalsupplyvoltage_dc__attribute_shortname,part_specs_nominalsupplyvoltage_dc__display_value,part_specs_operatingsupplyvoltage_attribute_id,part_specs_operatingsupplyvoltage_attribute_name,part_specs_operatingsupplyvoltage_attribute_shortname,part_specs_operatingsupplyvoltage_display_value,part_specs_militarystandard_attribute_id,part_specs_militarystandard_attribute_name,part_specs_militarystandard_attribute_shortname,part_specs_militarystandard_display_value,part_specs_dropoutvoltage_attribute_id,part_specs_dropoutvoltage_attribute_name,part_specs_dropoutvoltage_attribute_shortname,part_specs_dropoutvoltage_display_value,part_specs_terminalwidth_attribute_id,part_specs_terminalwidth_attribute_name,part_specs_terminalwidth_attribute_shortname,part_specs_terminalwidth_display_value,part_specs_elementconfiguration_attribute_id,part_specs_elementconfiguration_attribute_name,part_specs_elementconfiguration_attribute_shortname,part_specs_elementconfiguration_display_value,part_specs_maxrepetitivereversevoltage_vrrm__attribute_id,part_specs_maxrepetitivereversevoltage_vrrm__attribute_name,part_specs_maxrepetitivereversevoltage_vrrm__attribute_shortname,part_specs_maxrepetitivereversevoltage_vrrm__display_value,part_specs_peaknon_repetitivesurgecurrent_attribute_id,part_specs_peaknon_repetitivesurgecurrent_attribute_name,part_specs_peaknon_repetitivesurgecurrent_attribute_shortname,part_specs_peaknon_repetitivesurgecurrent_display_value,part_specs_peakreversecurrent_attribute_id,part_specs_peakreversecurrent_attribute_name,part_specs_peakreversecurrent_attribute_shortname,part_specs_peakreversecurrent_display_value,part_specs_databuswidth_attribute_id,part_specs_databuswidth_attribute_name,part_specs_databuswidth_attribute_shortname,part_specs_databuswidth_display_value,part_specs_maxfrequency_attribute_id,part_specs_maxfrequency_attribute_name,part_specs_maxfrequency_attribute_shortname,part_specs_maxfrequency_display_value,part_specs_ramsize_attribute_id,part_specs_ramsize_attribute_name,part_specs_ramsize_attribute_shortname,part_specs_ramsize_display_value,part_specs_inductance_attribute_id,part_specs_inductance_attribute_name,part_specs_inductance_attribute_shortname,part_specs_inductance_display_value,part_specs_gender_attribute_id,part_specs_gender_attribute_name,part_specs_gender_attribute_shortname,part_specs_gender_display_value,part_specs_speedgrade_attribute_id,part_specs_speedgrade_attribute_name,part_specs_speedgrade_attribute_shortname,part_specs_speedgrade_display_value,part_specs_housingmaterial_attribute_id,part_specs_housingmaterial_attribute_name,part_specs_housingmaterial_attribute_shortname,part_specs_housingmaterial_display_value,part_specs_flammabilityrating_attribute_id,part_specs_flammabilityrating_attribute_name,part_specs_flammabilityrating_attribute_shortname,part_specs_flammabilityrating_display_value,part_specs_numberofpositions_attribute_id,part_specs_numberofpositions_attribute_name,part_specs_numberofpositions_attribute_shortname,part_specs_numberofpositions_display_value,part_specs_ambienttemperaturerangehigh_attribute_id,part_specs_ambienttemperaturerangehigh_attribute_name,part_specs_ambienttemperaturerangehigh_attribute_shortname,part_specs_ambienttemperaturerangehigh_display_value,part_specs_frequencystability_attribute_id,part_specs_frequencystability_attribute_name,part_specs_frequencystability_attribute_shortname,part_specs_frequencystability_display_value,part_specs_corematerial_attribute_id,part_specs_corematerial_attribute_name,part_specs_corematerial_attribute_shortname,part_specs_corematerial_display_value,part_specs_seriesresistance_attribute_id,part_specs_seriesresistance_attribute_name,part_specs_seriesresistance_attribute_shortname,part_specs_seriesresistance_display_value,part_specs_esdprotection_attribute_id,part_specs_esdprotection_attribute_name,part_specs_esdprotection_attribute_shortname,part_specs_esdprotection_display_value,part_specs_numberofchannels_attribute_id,part_specs_numberofchannels_attribute_name,part_specs_numberofchannels_attribute_shortname,part_specs_numberofchannels_display_value,part_specs_life_cycles__attribute_id,part_specs_life_cycles__attribute_name,part_specs_life_cycles__attribute_shortname,part_specs_life_cycles__display_value,part_specs_cablelength_attribute_id,part_specs_cablelength_attribute_name,part_specs_cablelength_attribute_shortname,part_specs_cablelength_display_value,part_specs_connectortype_attribute_id,part_specs_connectortype_attribute_name,part_specs_connectortype_attribute_shortname,part_specs_connectortype_display_value,part_specs_wire_cabletype_attribute_id,part_specs_wire_cabletype_attribute_name,part_specs_wire_cabletype_attribute_shortname,part_specs_wire_cabletype_display_value,part_specs_commonmoderejectionratio_attribute_id,part_specs_commonmoderejectionratio_attribute_name,part_specs_commonmoderejectionratio_attribute_shortname,part_specs_commonmoderejectionratio_display_value,part_specs_gainbandwidthproduct_attribute_id,part_specs_gainbandwidthproduct_attribute_name,part_specs_gainbandwidthproduct_attribute_shortname,part_specs_gainbandwidthproduct_display_value,part_specs_voltagegain_attribute_id,part_specs_voltagegain_attribute_name,part_specs_voltagegain_attribute_shortname,part_specs_voltagegain_display_value,part_specs_loadcurrent_attribute_id,part_specs_loadcurrent_attribute_name,part_specs_loadcurrent_attribute_shortname,part_specs_loadcurrent_display_value,part_specs_shielding_attribute_id,part_specs_shielding_attribute_name,part_specs_shielding_attribute_shortname,part_specs_shielding_display_value,part_specs_dccurrent_attribute_id,part_specs_dccurrent_attribute_name,part_specs_dccurrent_attribute_shortname,part_specs_dccurrent_display_value,part_specs_saturationcurrent_attribute_id,part_specs_saturationcurrent_attribute_name,part_specs_saturationcurrent_attribute_shortname,part_specs_saturationcurrent_display_value,part_specs_maxsatcurrent_attribute_id,part_specs_maxsatcurrent_attribute_name,part_specs_maxsatcurrent_attribute_shortname,part_specs_maxsatcurrent_display_value,part_specs_maxtemprisecurrent_attribute_id,part_specs_maxtemprisecurrent_attribute_name,part_specs_maxtemprisecurrent_attribute_shortname,part_specs_maxtemprisecurrent_display_value,part_specs_rmscurrent_irms__attribute_id,part_specs_rmscurrent_irms__attribute_name,part_specs_rmscurrent_irms__attribute_shortname,part_specs_rmscurrent_irms__display_value,part_specs_temprisecurrent_attribute_id,part_specs_temprisecurrent_attribute_name,part_specs_temprisecurrent_attribute_shortname,part_specs_temprisecurrent_display_value,part_specs_countryoforigin_attribute_id,part_specs_countryoforigin_attribute_name,part_specs_countryoforigin_attribute_shortname,part_specs_countryoforigin_display_value,part_specs_eccncode_attribute_id,part_specs_eccncode_attribute_name,part_specs_eccncode_attribute_shortname,part_specs_eccncode_display_value,part_specs_htscode_attribute_id,part_specs_htscode_attribute_name,part_specs_htscode_attribute_shortname,part_specs_htscode_display_value,part_specs_numberoflines_attribute_id,part_specs_numberoflines_attribute_name,part_specs_numberoflines_attribute_shortname,part_specs_numberoflines_display_value,part_specs_inductance_parallel_attribute_id,part_specs_inductance_parallel_attribute_name,part_specs_inductance_parallel_attribute_shortname,part_specs_inductance_parallel_display_value,part_specs_inductance_series_attribute_id,part_specs_inductance_series_attribute_name,part_specs_inductance_series_attribute_shortname,part_specs_inductance_series_display_value,part_specs_numberofcoils_attribute_id,part_specs_numberofcoils_attribute_name,part_specs_numberofcoils_attribute_shortname,part_specs_numberofcoils_display_value,part_specs_conductormaterial_attribute_id,part_specs_conductormaterial_attribute_name,part_specs_conductormaterial_attribute_shortname,part_specs_conductormaterial_display_value,part_specs_dcresistance_dcr__series_attribute_id,part_specs_dcresistance_dcr__series_attribute_name,part_specs_dcresistance_dcr__series_attribute_shortname,part_specs_dcresistance_dcr__series_display_value,part_specs_bodymaterial_attribute_id,part_specs_bodymaterial_attribute_name,part_specs_bodymaterial_attribute_shortname,part_specs_bodymaterial_display_value,part_specs_turnsratio_attribute_id,part_specs_turnsratio_attribute_name,part_specs_turnsratio_attribute_shortname,part_specs_turnsratio_display_value,part_specs_coilresistance_attribute_id,part_specs_coilresistance_attribute_name,part_specs_coilresistance_attribute_shortname,part_specs_coilresistance_display_value,part_specs_maxvoltagerating_ac__attribute_id,part_specs_maxvoltagerating_ac__attribute_name,part_specs_maxvoltagerating_ac__attribute_shortname,part_specs_maxvoltagerating_ac__display_value,part_specs_approvals_attribute_id,part_specs_approvals_attribute_name,part_specs_approvals_attribute_shortname,part_specs_approvals_display_value,part_specs_rowspacing_attribute_id,part_specs_rowspacing_attribute_name,part_specs_rowspacing_attribute_shortname,part_specs_rowspacing_display_value,part_specs_primarywinding_s__attribute_id,part_specs_primarywinding_s__attribute_name,part_specs_primarywinding_s__attribute_shortname,part_specs_primarywinding_s__display_value,part_specs_secondarycurrent_attribute_id,part_specs_secondarycurrent_attribute_name,part_specs_secondarycurrent_attribute_shortname,part_specs_secondarycurrent_display_value,part_specs_testvoltage_attribute_id,part_specs_testvoltage_attribute_name,part_specs_testvoltage_attribute_shortname,part_specs_testvoltage_display_value,part_specs_busdirectional_attribute_id,part_specs_busdirectional_attribute_name,part_specs_busdirectional_attribute_shortname,part_specs_busdirectional_display_value,part_specs_density_attribute_id,part_specs_density_attribute_name,part_specs_density_attribute_shortname,part_specs_density_display_value,part_specs_datarate_attribute_id,part_specs_datarate_attribute_name,part_specs_datarate_attribute_shortname,part_specs_datarate_display_value,part_specs_logicfunction_attribute_id,part_specs_logicfunction_attribute_name,part_specs_logicfunction_attribute_shortname,part_specs_logicfunction_display_value,part_specs_isolationvoltage_attribute_id,part_specs_isolationvoltage_attribute_name,part_specs_isolationvoltage_attribute_shortname,part_specs_isolationvoltage_display_value,part_specs_mscp_attribute_id,part_specs_mscp_attribute_name,part_specs_mscp_attribute_shortname,part_specs_mscp_display_value,part_specs_inputcurrent_attribute_id,part_specs_inputcurrent_attribute_name,part_specs_inputcurrent_attribute_shortname,part_specs_inputcurrent_display_value,part_specs_numberofturns_attribute_id,part_specs_numberofturns_attribute_name,part_specs_numberofturns_attribute_shortname,part_specs_numberofturns_display_value,part_specs_numberofprogrammablei_o_attribute_id,part_specs_numberofprogrammablei_o_attribute_name,part_specs_numberofprogrammablei_o_attribute_shortname,part_specs_numberofprogrammablei_o_display_value,part_specs_numberoftimers_counters_attribute_id,part_specs_numberoftimers_counters_attribute_name,part_specs_numberoftimers_counters_attribute_shortname,part_specs_numberoftimers_counters_display_value,part_specs_watchdogtimer_attribute_id,part_specs_watchdogtimer_attribute_name,part_specs_watchdogtimer_attribute_shortname,part_specs_watchdogtimer_display_value,part_specs_addressbuswidth_attribute_id,part_specs_addressbuswidth_attribute_name,part_specs_addressbuswidth_attribute_shortname,part_specs_addressbuswidth_display_value,part_specs_numberofports_attribute_id,part_specs_numberofports_attribute_name,part_specs_numberofports_attribute_shortname,part_specs_numberofports_display_value,part_specs_evaluationkit_attribute_id,part_specs_evaluationkit_attribute_name,part_specs_evaluationkit_attribute_shortname,part_specs_evaluationkit_display_value"
cols = cols.split(',')
print(cols)


['Unnamed: 0', '_cache_id', 'description', 'part__cache_id', 'part_best_datasheet_url', 'part_best_image_url', 'part_category_id', 'part_counts_companion_products', 'part_counts_datasheets', 'part_counts_descriptions', 'part_counts_images', 'part_counts_reference_designs', 'part_counts_similar_parts', 'part_counts_specs', 'part_descriptions_0_text', 'part_descriptions_1_text', 'part_descriptions_2_text', 'part_descriptions_3_text', 'part_id', 'part_manufacturer_id', 'part_manufacturer_is_verified', 'part_manufacturer_name', 'part_manufacturer_url', 'part_median_price_1000__cache_id', 'part_median_price_1000_converted_currency', 'part_median_price_1000_converted_price', 'part_mpn', 'part_specs_capacitance_attribute_id', 'part_specs_capacitance_attribute_name', 'part_specs_capacitance_attribute_shortname', 'part_specs_capacitance_display_value', 'part_specs_case_package_attribute_id', 'part_specs_case_package_attribute_name', 'part_specs_case_package_attribute_shortname', 'part_specs_cas

In [3]:
host = "ec2-34-233-115-14.compute-1.amazonaws.com"
port = 5432
database = "dfu56m15dkhh46"
user = "pgyrjmstmyerfk"
password = "228fcbba14e9d2bf362fcaa29cabe1106cc8dba00605f45ee25e810194309fd4"

conn_str = f"postgresql://{user}:{password}@{host}:{port}/{database}"
local_str = 'postgresql://shalinshah@localhost:5432/postgres'

In [6]:
from sqlalchemy import create_engine, event
import pandas as pd

engine = create_engine(conn_str)

@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True
        cursor.commit()
        
df = pd.read_csv('csv/all_2.csv')
# print(df.head())
df.to_sql('passive_components', engine, schema='public', if_exists='replace', index=False, method="multi")


/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_67904/3999386347.py:12: DtypeWarning: Columns (17,18,23,29,30,31,35,37,38,39,41,42,43,49,50,51,53,54,55,57,58,59,61,62,63,73,74,75,77,78,79,81,82,83,85,86,87,93,94,95,97,98,99,101,102,103,105,106,107,109,110,111,113,114,115,117,118,120,122,123,124,128,129,130,131,138,139,140,142,143,144,146,147,148,150,151,152,154,155,156,158,159,160,162,163,164,166,167,168,170,171,172,174,175,176,178,179,180,182,183,184,186,187,188,189,191,192,193,195,196,197,199,200,201,203,204,205,207,208,209,211,212,213,215,216,217,219,220,221,222,224,225,226,228,229,230,232,233,234,236,237,240,241,244,245,246,247,249,250,251,253,254,255,257,258,259,261,262,265,266,267,269,270,271,273,274,275,277,278,279,281,282,283,285,286,287,289,290,291,293,294,295,297,298,299,300,302,303,304,306,307,308,310,311,312,314,315,316,318,319,320,322,323,324,326,327,328,330,331,332,334,335,336,338,339,340,342,343,344,345,346,347,349,350,351,353,354,355,357,358,359,361,362,363,3

   Unnamed: 0.1  Unnamed: 0        _cache_id  \
0             0           0   US-USD-7717440   
1             1           1   US-USD-4533279   
2             2           2   US-USD-4533571   
3             3           3  US-USD-81248714   
4             4           4  US-USD-26300285   

                                         description   part__cache_id  \
0  Aluminum Electrolytic Capacitors - Screw Termi...   US-USD-7717440   
1  Aluminum Electrolytic Capacitors - Screw Termi...   US-USD-4533279   
2  Aluminum Electrolytic Capacitors - Screw Termi...   US-USD-4533571   
3  Capacitors, Aluminum Electrolytic, Screw Termi...  US-USD-81248714   
4  Aluminum Electrolytic Capacitors - Screw Termi...  US-USD-26300285   

                             part_best_datasheet_url  \
0  http://datasheet.octopart.com/HES172G450W4C-Co...   
1  http://datasheet.octopart.com/CGH172T450W3L-Co...   
2  http://datasheet.octopart.com/CGS172T450W3L-Co...   
3  http://datasheet.octopart.com/ALS80A133DA040-

In [10]:
import d6tstack
df = pd.read_csv('csv/all_2.csv')
# df.drop(["Unnamed: 0"], axis = 1, inplace = True)
d6tstack.utils.pd_to_psql(df, f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}', 'passive_components', if_exists='replace')




/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_67904/3103723258.py:2: DtypeWarning: Columns (1,7,8,9,10,11,12,13,14,17,18,19,20,21,23,26,28,29,30,31,32,35,36,37,38,39,40,41,42,43,44,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,68,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,2

True

In [3]:
ambiguous_cols = [17,18,23,29,30,31,35,37,38,39,41,42,43,49,50,51,53,54,55,57,58,59,61,62,63,73,74,75,77,78,79,81,82,83,85,86,87,93,94,95,97,98,99,101,102,103,105,106,107,109,110,111,113,114,115,117,118,120,122,123,124,128,129,130,131,138,139,140,142,143,144,146,147,148,150,151,152,154,155,156,158,159,160,162,163,164,166,167,168,170,171,172,174,175,176,178,179,180,182,183,184,186,187,188,189,191,192,193,195,196,197,199,200,201,203,204,205,207,208,209,211,212,213,215,216,217,219,220,221,222,224,225,226,228,229,230,232,233,234,236,237,240,241,244,245,246,247,249,250,251,253,254,255,257,258,259,261,262,265,266,267,269,270,271,273,274,275,277,278,279,281,282,283,285,286,287,289,290,291,293,294,295,297,298,299,300,302,303,304,306,307,308,310,311,312,314,315,316,318,319,320,322,323,324,326,327,328,330,331,332,334,335,336,338,339,340,342,343,344,345,346,347,349,350,351,353,354,355,357,358,359,361,362,363,365,366,367,369,370,371,373,374,375,377,378,381,382,385,386,389,390,391,393,394,395,397,398,401,402,403,405,406,407,409,410,411,413,414,415,417,418,419,421,422,423,425,426,427,429,430,431,433,434,435,437,438,439,441,442,443,445,446,447,449,450,451,453,454,457,458,459,461,462,463,465,466,469,470,471,473,474,475,477,478,479,481,482,483,485,486,487,489,490,491,493,494,495,497,498,499,501,502,503,505,506,507,509,510,511,513,514,515,517,518,519,521,522,523,525,526,527,529,530,531,533,534,535,537,538,539,541,542,543,545,546,547,549,550,551,553,554,555,557,558,559,561,562,563,565,566,567,569,570,571,573,574,575,577,578,579,581,582,585,586,587,589,590,591,593,594,597,598,599,601,602,603,605,606,607,609,610,611,613,614,615,617,618,621,622,623,625,626,627,629,630,631,633,634,635,637,638,639,641,642,643,645,646,647,649,650,651,653,654,655,657,658,659,661,662,663,665,666,667,669,670,671,673,674,675,677,678,679,681,682,683,685,686,687,689,690,691,693,694,697,698,699,701,702,703,705,706,709,710,711,713,714,715,717,718,719,721,722,723,725,726,727,729,730,731,733,734,735,737,738,739,741,742,743,745,746,747,749,750,751,753,754,755,757,758,759,761,762,763,765,766,767,769,770,771,773,774,775,777,778,779,781,782,785,786,789,790,793,794,795,797,798,799,801,802,805,806,807]

df = pd.read_csv('csv/mica.csv')






/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_14736/3845151789.py:3: DtypeWarning: Columns (183,184,185) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('csv/mica.csv')


In [22]:
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''

df = pd.read_csv('csv/all.csv')
l = list(df)
mapping = {}
ambiguous_cols = [17,18,23,29,30,31,35,37,38,39,41,42,43,49,50,51,53,54,55,57,58,59,61,62,63,73,74,75,77,78,79,81,82,83,85,86,87,93,94,95,97,98,99,101,102,103,105,106,107,109,110,111,113,114,115,117,118,120,122,123,124,128,129,130,131,138,139,140,142,143,144,146,147,148,150,151,152,154,155,156,158,159,160,162,163,164,166,167,168,170,171,172,174,175,176,178,179,180,182,183,184,186,187,188,189,191,192,193,195,196,197,199,200,201,203,204,205,207,208,209,211,212,213,215,216,217,219,220,221,222,224,225,226,228,229,230,232,233,234,236,237,240,241,244,245,246,247,249,250,251,253,254,255,257,258,259,261,262,265,266,267,269,270,271,273,274,275,277,278,279,281,282,283,285,286,287,289,290,291,293,294,295,297,298,299,300,302,303,304,306,307,308,310,311,312,314,315,316,318,319,320,322,323,324,326,327,328,330,331,332,334,335,336,338,339,340,342,343,344,345,346,347,349,350,351,353,354,355,357,358,359,361,362,363,365,366,367,369,370,371,373,374,375,377,378,381,382,385,386,389,390,391,393,394,395,397,398,401,402,403,405,406,407,409,410,411,413,414,415,417,418,419,421,422,423,425,426,427,429,430,431,433,434,435,437,438,439,441,442,443,445,446,447,449,450,451,453,454,457,458,459,461,462,463,465,466,469,470,471,473,474,475,477,478,479,481,482,483,485,486,487,489,490,491,493,494,495,497,498,499,501,502,503,505,506,507,509,510,511,513,514,515,517,518,519,521,522,523,525,526,527,529,530,531,533,534,535,537,538,539,541,542,543,545,546,547,549,550,551,553,554,555,557,558,559,561,562,563,565,566,567,569,570,571,573,574,575,577,578,579,581,582,585,586,587,589,590,591,593,594,597,598,599,601,602,603,605,606,607,609,610,611,613,614,615,617,618,621,622,623,625,626,627,629,630,631,633,634,635,637,638,639,641,642,643,645,646,647,649,650,651,653,654,655,657,658,659,661,662,663,665,666,667,669,670,671,673,674,675,677,678,679,681,682,683,685,686,687,689,690,691,693,694,697,698,699,701,702,703,705,706,709,710,711,713,714,715,717,718,719,721,722,723,725,726,727,729,730,731,733,734,735,737,738,739,741,742,743,745,746,747,749,750,751,753,754,755,757,758,759,761,762,763,765,766,767,769,770,771,773,774,775,777,778,779,781,782,785,786,789,790,793,794,795,797,798,799,801,802,805,806,807]
for col in ambiguous_cols:
    mapping[l[col]] = convert_dtype
print(mapping)

/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_14736/4023270257.py:9: DtypeWarning: Columns (17,18,23,29,30,31,35,37,38,39,41,42,43,49,50,51,53,54,55,57,58,59,61,62,63,73,74,75,77,78,79,81,82,83,85,86,87,93,94,95,97,98,99,101,102,103,105,106,107,109,110,111,113,114,115,117,118,120,122,123,124,128,129,130,131,138,139,140,142,143,144,146,147,148,150,151,152,154,155,156,158,159,160,162,163,164,166,167,168,170,171,172,174,175,176,178,179,180,182,183,184,186,187,188,189,191,192,193,195,196,197,199,200,201,203,204,205,207,208,209,211,212,213,215,216,217,219,220,221,222,224,225,226,228,229,230,232,233,234,236,237,240,241,244,245,246,247,249,250,251,253,254,255,257,258,259,261,262,265,266,267,269,270,271,273,274,275,277,278,279,281,282,283,285,286,287,289,290,291,293,294,295,297,298,299,300,302,303,304,306,307,308,310,311,312,314,315,316,318,319,320,322,323,324,326,327,328,330,331,332,334,335,336,338,339,340,342,343,344,345,346,347,349,350,351,353,354,355,357,358,359,361,362,363,36

{'part_descriptions_2_text': <function convert_dtype at 0x1531b3a60>, 'part_descriptions_3_text': <function convert_dtype at 0x1531b3a60>, 'part_manufacturer_url': <function convert_dtype at 0x1531b3a60>, 'part_specs_capacitance_attribute_name': <function convert_dtype at 0x1531b3a60>, 'part_specs_capacitance_attribute_shortname': <function convert_dtype at 0x1531b3a60>, 'part_specs_capacitance_display_value': <function convert_dtype at 0x1531b3a60>, 'part_specs_case_package_display_value': <function convert_dtype at 0x1531b3a60>, 'part_specs_diameter_attribute_name': <function convert_dtype at 0x1531b3a60>, 'part_specs_diameter_attribute_shortname': <function convert_dtype at 0x1531b3a60>, 'part_specs_diameter_display_value': <function convert_dtype at 0x1531b3a60>, 'part_specs_esr_equivalentseriesresistance__attribute_name': <function convert_dtype at 0x1531b3a60>, 'part_specs_esr_equivalentseriesresistance__attribute_shortname': <function convert_dtype at 0x1531b3a60>, 'part_specs_e

In [7]:
df_2 = pd.read_csv('csv/all.csv', dtype=mapping)

In [8]:
df_2.to_csv("csv/all_10.csv")

In [12]:
df3 = pd.read_csv('csv/all_10.csv', converters=mapping)
df3.drop(["a"], axis = 1, inplace = True)
df3.drop(["a_2"], axis = 1, inplace = True)
df3.to_csv("csv/all_f.csv")

In [26]:
df8 = pd.read_csv('csv/all.csv')

/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_14736/3025125821.py:1: DtypeWarning: Columns (17,18,23,29,30,31,35,37,38,39,41,42,43,49,50,51,53,54,55,57,58,59,61,62,63,73,74,75,77,78,79,81,82,83,85,86,87,93,94,95,97,98,99,101,102,103,105,106,107,109,110,111,113,114,115,117,118,120,122,123,124,128,129,130,131,138,139,140,142,143,144,146,147,148,150,151,152,154,155,156,158,159,160,162,163,164,166,167,168,170,171,172,174,175,176,178,179,180,182,183,184,186,187,188,189,191,192,193,195,196,197,199,200,201,203,204,205,207,208,209,211,212,213,215,216,217,219,220,221,222,224,225,226,228,229,230,232,233,234,236,237,240,241,244,245,246,247,249,250,251,253,254,255,257,258,259,261,262,265,266,267,269,270,271,273,274,275,277,278,279,281,282,283,285,286,287,289,290,291,293,294,295,297,298,299,300,302,303,304,306,307,308,310,311,312,314,315,316,318,319,320,322,323,324,326,327,328,330,331,332,334,335,336,338,339,340,342,343,344,345,346,347,349,350,351,353,354,355,357,358,359,361,362,363,36

In [27]:
for k in mapping:
    df8[str(k)] = df8[str(k)].map(str)
# df3.drop(["a"], axis = 1, inplace = True)
# df3.drop(["a_2"], axis = 1, inplace = True)
df8.to_csv("csv/all_df8.csv")


In [29]:
df9 = pd.read_csv('csv/all_df8.csv')

/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_14736/62140523.py:1: DtypeWarning: Columns (18,19,24,30,31,32,36,38,39,40,42,43,44,50,51,52,54,55,56,58,59,60,62,63,64,74,75,76,78,79,80,82,83,84,86,87,88,94,95,96,98,99,100,102,103,104,106,107,108,110,111,112,114,115,116,118,119,121,123,124,125,129,130,131,132,139,140,141,143,144,145,147,148,149,151,152,153,155,156,157,159,160,161,163,164,165,167,168,169,171,172,173,175,176,177,179,180,181,183,184,185,187,188,189,190,192,193,194,196,197,198,200,201,202,204,205,206,208,209,210,212,213,214,216,217,218,220,221,222,223,225,226,227,229,230,231,233,234,235,237,238,241,242,245,246,247,248,250,251,252,254,255,256,258,259,260,262,263,266,267,268,270,271,272,274,275,276,278,279,280,282,283,284,286,287,288,290,291,292,294,295,296,298,299,300,301,303,304,305,307,308,309,311,312,313,315,316,317,319,320,321,323,324,325,327,328,329,331,332,333,335,336,337,339,340,341,343,344,345,346,347,348,350,351,352,354,355,356,358,359,360,362,363,364,366